In [9]:
import wandb
import torch
# from utils2 import eval_zs_gzsl
from helper_func import eval_zs_gzsl
from dataset_test import SUNDataLoader
import argparse
import json
import pickle
import os
import pickle
from torchvision import transforms
from torch.utils.data import Dataset, Subset, DataLoader
from PIL import Image
import sys

import torch.optim as optim
import torch.nn as nn
import numpy as np
from model_test import BDA
# from dataset import AWA2DataLoader
from  helper_func import eval_zs_gzsl
import json
import argparse

In [10]:
dataloader = SUNDataLoader('.', "cuda:0")

.
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
SUN
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
Balance dataloader
_____
./../revice_Transzero/data/SUN/feature_map_ResNet_101_SUN.hdf5
Expert Attr


In [14]:
sweep_configuration = {
    'method': 'grid',
    'name': ' test_sun_gzsl',
    'metric': {'goal': 'maximize', 'name': 'best_H'},
    'parameters': 
    {   'batch_size':    {'value': 50},
        'dataset': { 'value': 'SUN'},
        'device': {'value': 'cuda:0'},
        'dim_f': { 'value': 2048},
        'dim_v':{ 'value': 300},
        'epochs': {'value': 250},
        'img_size': {'value': 448},
        'lambda_': {'value': 0.1},
        'lambda_reg': {'value': 0.05},
        'lambda_CEVi': {'value':  1},
        'normalize_V': {'value': "False"},
        'num_attribute': { 'value': 102},
        'num_class': {'value': 717},
        'random_seed': {'value': 6},
        'tf_SAtt':  {'value': "false"},
        'tf_aux_embed': {'value': "True"},
        'tf_common_dim': {'value': 350},
        'tf_dc_layer': {'value': 1},
        'tf_dim_feedforward': {'value': 2048},
        'tf_dropout': {'value': 0.35},
        'tf_dropout1': {'value': 1},       
        
        'tf_ec_layer': {'value': 1},
        'tf_heads': {'value': 1},
        'alpha': {'value': 1},
        'beta' : {'value': 1},
        'fsqrt': {'value': 128},
        'sqrt': {'value':  130},
        'weight' : {'values': [1]},
        'weight_self' : {'values': [1]},
        'tgt_weight' : {'values': [1]}                         
        } 
}
                   
sweep_id = wandb.sweep(sweep=sweep_configuration, project='Ko_bda')

Create sweep with ID: f4kzjg33
Sweep URL: https://wandb.ai/junseoklee_333/Ko_bda/sweeps/f4kzjg33


In [15]:
def main():
    run = wandb.init()
    config = wandb.config
    # set random seed
    seed = config.random_seed
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)

    # TransZero model
    model = BDA(config, dataloader.att, dataloader.w2v_att,
                    dataloader.seenclasses, dataloader.unseenclasses).to(config.device)    
    
    # main loop
    niters = dataloader.ntrain * config.epochs//config.batch_size
    report_interval = niters//config.epochs
    best_performance = [0, 0, 0, 0]
    best_performance_zsl = 0
    model_dict = model.state_dict()
    # saved_dict = torch.load(config.saved_model)
    check_point = torch.load("./checkpoint/SUN/SUN_gzsl.pth")
    # saved_dict = check_point["model_state_dict"]
    saved_dict = {k: v for k, v in check_point.items() if k in model_dict}
    model_dict.update(saved_dict)
    model.load_state_dict(model_dict)
    model.to(config.device)
    model.eval()

    # report result

    print('-'*30)
    acc_seen, acc_novel, H, acc_zs = eval_zs_gzsl(
        dataloader, model, config.device, bias_seen=0, bias_unseen=0)

    if H > best_performance[2]:
        best_performance = [acc_novel, acc_seen, H, acc_zs]

    print('acc_unseen=%.3f, acc_seen=%.3f, H=%.3f | '
        'acc_zs=%.3f' % (  best_performance[0], best_performance[1],
            best_performance[2], best_performance_zsl))

    wandb.log({
        'best_acc_unseen': best_performance[0],
        'best_acc_seen': best_performance[1],
        'best_H': best_performance[2]
        })

In [16]:
wandb.agent(sweep_id, function=main, count=2 )

wandb: Agent Starting Run: wbezsxa7 with config:
wandb: 	alpha: 1
wandb: 	batch_size: 50
wandb: 	beta: 1
wandb: 	dataset: SUN
wandb: 	device: cuda:0
wandb: 	dim_f: 2048
wandb: 	dim_v: 300
wandb: 	epochs: 250
wandb: 	fsqrt: 128
wandb: 	img_size: 448
wandb: 	lambda_: 0.1
wandb: 	lambda_CEVi: 1
wandb: 	lambda_reg: 0.05
wandb: 	normalize_V: False
wandb: 	num_attribute: 102
wandb: 	num_class: 717
wandb: 	random_seed: 6
wandb: 	sqrt: 130
wandb: 	tf_SAtt: false
wandb: 	tf_aux_embed: True
wandb: 	tf_common_dim: 350
wandb: 	tf_dc_layer: 1
wandb: 	tf_dim_feedforward: 2048
wandb: 	tf_dropout: 0.35
wandb: 	tf_dropout1: 1
wandb: 	tf_ec_layer: 1
wandb: 	tf_heads: 1
wandb: 	tgt_weight: 1
wandb: 	weight: 1
wandb: 	weight_self: 1
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


------------------------------
acc_unseen=0.552, acc_seen=0.327, H=0.411 | acc_zs=0.000


best_H,▁
best_acc_seen,▁
best_acc_unseen,▁
best_H,0.41052
best_acc_seen,0.32674
best_acc_unseen,0.55208


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
